# Данные

In [1]:
import csv
from collections import defaultdict
import random
import json
random.seed(42)

In [2]:
import sklearn.ensemble
import numpy as np

In [3]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [4]:
#Словари для основной выборки
user_to_items = defaultdict(set)
item_to_users = defaultdict(set)

#Словари для тестовой выборки
test_user_to_items = defaultdict(set)
test_item_to_users = defaultdict(set)



with open("data/train_likes.csv") as datafile:
    for like in csv.DictReader(datafile):
        # Кидаем монетку. В зависимости от результата кладём в обучение или тест
        if random.random() < 0.90:
            user_to_items[like['user_id']].add(like['item_id'])
            item_to_users[like['item_id']].add(like['user_id'])
        else:
            test_user_to_items[like['user_id']].add(like['item_id'])
            test_item_to_users[like['item_id']].add(like['user_id'])

In [5]:
all_items = set(item_to_users.keys()) | set(test_item_to_users.keys())
all_users = set(user_to_items.keys()) | set(test_user_to_items.keys())

In [7]:
user_to_i = {user: i for i, user in enumerate(all_users)}
item_to_i = {item: i for i, item in enumerate(all_items)}
all_users_list = list(all_users)
all_items_list = list(all_items)

In [8]:
matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
test_matrix = sp.sparse.lil_matrix((len(all_users), len(all_items)))
for user, items in user_to_items.items():
    for item in items:
        matrix[user_to_i[user], item_to_i[item]] = True
for user, items in test_user_to_items.items():
    for item in items:
        test_matrix[user_to_i[user], item_to_i[item]] = True
matrix = matrix.tocsr()
test_matrix = test_matrix.tocsr()

In [9]:
matrix

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 99582 stored elements in Compressed Sparse Row format>

In [27]:
u, s, vt = sp.sparse.linalg.svds(matrix.astype(np.float32), k=100)

In [30]:
u, s, vt

(array([[  1.17884148e-04,  -5.88815368e-04,   1.13425357e-03, ...,
          -2.03856631e-04,  -1.18520089e-04,   1.51377928e-04],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  1.01056103e-05,   1.02588126e-06,   4.77186268e-06, ...,
          -2.55169311e-07,  -1.22939795e-07,   1.46044172e-07],
        ..., 
        [  1.30094704e-03,  -2.54712836e-03,   3.55348131e-03, ...,
          -1.12759310e-03,  -7.75150023e-04,   1.05613959e-03],
        [ -3.43474501e-04,  -2.56756030e-04,  -4.53010376e-04, ...,
           4.41513304e-03,   2.50009149e-02,   1.90571472e-02],
        [  1.63380059e-10,  -1.30901179e-10,  -1.08317418e-10, ...,
          -3.32123183e-11,  -2.60462849e-11,  -3.29792617e-11]], dtype=float32),
 array([ 10.72060871,  10.74181652,  10.77247047,  10.81643772,
         10.87412643,  10.89381599,  10.90676785,  10.93737316,
         11.00072193,  11.02944088,  11.0499773 ,

# Фильтрация пользователей
* Значительная часть пользователей имеет всего 1-2 просмотра. При всём желании, рекоммендовать им что-либо осмысленное при помощи рассматриваемого здесь метода мы вряд ли сможем. Для простоты вычислений, удалим их из выборки.
* Важно понимать, что качество на оставшихся пользователях скорее всего будет выше, чем на первоначальной выборке.

In [11]:
min_items_per_user = 2
from copy import copy
for user in copy(test_user_to_items).keys():
    
    n_items_per_user = len(user_to_items[user]) + len(test_user_to_items[user])
    
    if n_items_per_user <= min_items_per_user:
        del user_to_items[user]
        del test_user_to_items[user]

### Рекоммендующая функция
Позволим себе немного вольности: наша функция будет возвращать не вероятности, а список фильмов в порядке убывания "рекомендованности".

* Рекоммендованность фильма item пользователю user посчитаем так:
  * Для каждого фильма, полайканного пользователем user, найдём других людей, которым понравился фильм.
  * Сложим всех таких "друзей по лайкам" вместе и назовём соседями (__neighborhood__) пользователя.
  * Для фильма item узнаем его аудиторию - множество пользователей, которые его лайкнули
  * Пригодность фильма пользователю - то, насколько "друзьям по лайкам" пользователя нравится этот фильм.

Для примера, будем использовать косинусную меру расстояния
  
$ cos(u_{film}, u_{neighborhood}) = $ =$ u_{film} \cdot u_{neighborhood} \over |u_{film}| |u_{neighborhood}| $


$u_{neighborhood}$ зависит только от пользователя, но не от фильма, поэтому при сравнении фильмов по пригодности для одного пользователя, его можно исключить из формулы для простоты вычислений.

$ similarity(u_{film}, u_{neighborhood}) = $ $  u_{film} \cdot u_{neighborhood} \over |u_{film}| $
  
  
Распишем формулу подробно:

$ similarity(u_{film}, u_{neighborhood}) = $ $ \sum _{u_i} [u_i \in u_{film}] \cdot [u_i \in u_{neighborhood}] \over |u_{film}|  $

* u_i - очередной пользователь (в цикле по всем пользователям)
  
Выражение $[u_i \in u_{neighborhood}]$ здесь означает "сколько раз очередной пользователь входит в множество друзей по лайкам"
  
  

In [12]:
from math import sqrt
from collections import Counter

def recommend(user, n_best = 10):
    user_items = user_to_items[user]
    
    neighborhood = Counter()
    for item in user_items:
        neighborhood.update(item_to_users[item])
    
    #словарь {фильм -> пригодность фильма пользователю}
    item_similarities = {}
    
    for item in all_items:
        if item in user_items: continue
        item_users = item_to_users[item]
        if len(item_users) == 0: continue
        
        n_common_users = sum(neighborhood[user] for user in item_users)
        similarity = float(n_common_users) / sqrt(len(item_users))
        item_similarities[item] = similarity
    
    items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
    
    return items_sorted[:n_best]

In [13]:
user_to_int = dict()
for i, u in enumerate(all_users):
    user_to_int[u] = i 

In [14]:
films = json.load(open('data/items.json'))
films = {a['id']:a for a in films}

In [15]:
#Making dataset
user_features = defaultdict(lambda:defaultdict(lambda:0))
for u in list(user_to_items.keys())[0:]:
    for item in user_to_items[u]:
        if item in films:
            for feature, value in films[item].items():
                if feature != 'id':
                    if feature=='genre':
                        user_features[u][value] += 1
                    else:    
                        user_features[u][feature]+=value
    for f in user_features[u]:
        user_features[u][f]/=len(user_to_items)

In [52]:
def user_film_mk1(user, item, debug=False):
    #print(len(user_features[user]))
    cursum = 0
    curcnt = 0
    if item in films:
        for feature, value in films[item].items():
            if type(value) is int:
                curcnt+=1
                if feature=='genre':
                    #print(user_features[user])
                    cursum+=user_features[user][value]
                else:    
                    cursum+=value*user_features[user][feature]
    if curcnt==0:
        curcnt+=1
    #cursum/=curcnt
    if debug:
        print(cursum)
    return cursum

In [35]:
def user_film_mk2(user, item, debug=False):
    ui = user_to_i[user]
    ii = item_to_i[item]
    if debug:
        print(ui, ii)
    a = np.dot(u[ui,:] * s, vt[:,ii])
    return a

In [25]:
# Порекоммендуем топ-5 фильмов какому-то юзверю
recommend('d8c2794b01531ca807bc2b28d171f22d', n_best=5)

['44327280355abfc1c58fa9ad8c41a2cc',
 '1d621e6a95db7b8115d99d0589d6e71e',
 '8cb44b2217dd1cc509465a07cbb68152',
 'd5ba3ed09490eb5bc1b92a87ea99e227',
 'e6e53f41066b37fb5b80bd118dc800be']

In [32]:
user_film_mk2('17db499bbbd058da91bddcaea365be5c', '98bdfa4bcad95291e88d56521b620acc')

12310 16583


-2.5699296e-06

In [33]:
def generic_recommend(user_flim_function):
    def recommend(user, n_best = 10, debug=False):
        user_items = user_to_items[user]

        item_similarities = {}
        for item in all_items:
            if item in user_items: continue
            item_users = item_to_users[item]
            if len(item_users) == 0: continue

            item_similarities[item] = user_flim_function(user, item)

        items_sorted = sorted(all_items, key = lambda x: item_similarities.get(x, 0),reverse = True)
        if debug:
            for a in  items_sorted[:n_best]:
                print((a, item_similarities.get(a, 0)))
        return items_sorted[:n_best]
    return recommend

In [51]:
generic_recommend(user_film_mk2)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk2('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('7682d6641caf183ffcf2616c9bb1e434', 0.087393627)
('b5e424608d041f38c2fd6125b3bcd40f', 0.062321059)
('006a602774c79d476cdbd5dff7dd24f1', 0.047886699)
('e9caaa12a277abf87ba07b84e09a4170', 0.045560557)
('aa5f2ca699da42e467e550f9f071fb3f', 0.044250544)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.044186771)
('e8047b4262e676d28f50816ac3fde1ca', 0.042720884)
('77c7998b3d3d41f3fc3aec786b4015ac', 0.038098589)
('1da6f84598fbeea050da49fe37297932', 0.037306532)
('2dc12b93072f94bca9438902bddc36c4', 0.03517025)
1143 18545


0.0024644625

In [53]:
generic_recommend(user_film_mk1)('1a337111f63f6a7f86cebf6b2ad3d732', debug=True)
user_film_mk1('1a337111f63f6a7f86cebf6b2ad3d732', 'c745ed11b94ed93f3008897c75240de3', debug=True)

('e13df82b3252eaeefc0c13599695f9c7', 0.0006581965414763549)
('634e622e5c957c2cd8ddb98ca75ef937', 0.0005584697927678162)
('8faa6327a2ca2eb92e738ffc204d35c0', 0.0005584697927678162)
('64b6536649e20bd72d7183ec717b6c86', 0.0005385244430261085)
('5c6d1862dc1e82f340962b308999e603', 0.0005185790932844008)
('ef786a839c609561b76ce03768eace80', 0.0005185790932844007)
('bcee3a68dbfb6b4ea97ac53ceb1d3287', 0.0005185790932844007)
('ff987a60d68f5bbbefe97f8c5062711a', 0.0004986337435426931)
('753bce7ac492e6c95b07b85f8877cf5a', 0.0004786883938009853)
('7b0f1b0815292c0a550fabd5eeadc011', 0.00045874304405927767)
0


0

In [40]:
list(test_user_to_items.items())[1]

('1a337111f63f6a7f86cebf6b2ad3d732', {'c745ed11b94ed93f3008897c75240de3'})

In [41]:
def recommend_dummy(user, n_best = 10):
    item_similarities = []
    for item in all_items:
        #пропустим те фильмы, которые пользователь уже просмотрел, если нас об этом попросили
        if item in user_to_items[user]: continue
        item_similarities.append(item)
         
    random.shuffle(item_similarities)
    #вернём n_best наиболее пригодных
    #print(items_sorted[:n_best])
    return item_similarities[:n_best]

# Оценка качества - map@k

In [45]:
check_quality(recommend_dummy,10,200)

0 / 200
100 / 200
AP@10 = 0.0


In [46]:
check_quality(generic_recommend(user_film_mk1),10,200)

0 / 200
100 / 200
AP@10 = 0.00475079365079365


In [47]:
check_quality(generic_recommend(user_film_mk2),10,200)

0 / 200
100 / 200
AP@10 = 0.020573412698412695


In [48]:
check_quality(recommend,10,200)

0 / 200
100 / 200
AP@10 = 0.013998015873015872


In [49]:
len(test_user_to_items)

2732

In [43]:
def APatK (actual_likes,recommendation_list, K =10):
    """Посчитать Average Precision at K"""

    countRelevants = 0
    sum_of_precisions = 0.0
    
    for i in range(min(K,len(recommendation_list))):
        currentk = i + 1.0
        if recommendation_list[i] in actual_likes:
            countRelevants+=1
        precisionAtK = countRelevants / currentk 
        sum_of_precisions += precisionAtK
        
    return sum_of_precisions / K 

In [44]:
#сколько рекоммендаций рассматриваем
def check_quality(function, K = 10, max_n_users = 500):
    APatK_per_user = []
    user_list = list(test_user_to_items.keys())[:max_n_users]
    for i, user in enumerate(user_list):
        #фильмы, которые пользователю на самом деле нравятся
        test_items = test_user_to_items[user]

        #Выдать топ-K рекоммендаций
        recommendation_list = function(user,n_best=K)

        #Посчитать ap@k
        user_APatK = APatK(test_items, recommendation_list,K=K)

        #и сложить в коробку
        APatK_per_user.append(user_APatK)

        #Progress bar
        if i % 100 ==0:
            print(i,'/',max_n_users)

        if i > max_n_users:
            break

    print('AP@{} = {}'.format(K, sum(APatK_per_user)/len(APatK_per_user)))


# Notes
* Кроме качества рекоммендаций, map@k ещё зависит от доли тестовой выборки, фильтрации и от самого K. Сравнивать качество разных алгоритмов имеет смысл только при одинаковом K и тестовой выборке.
* Давать полезные рекоммендации пользователям с малым числом просмотров тоже можно: например, можно выдавать наиболее популярные в целом фильмы.
* Разделение на обучение/тест честнее делать на по времени: первые 70% (например) лайков в обучение, остальные в тест. Это ближе к реальной жизни, когда вы сначала обучаете модель на логах, а потом применяете на новых сессиях пользователей.